In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 10 17:34:18 2020

@author: Guillermo Sánchez Gutiérrez-Cabello
"""
#%%
import numpy as np
import pandas as pd
import os
import sklearn
#pd.options.mode.chained_assignment = None  # default='warn'


from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

import nltk
nltk.download("popular") # required to download the stopwords lists
from nltk.corpus import stopwords

spanish_stopwords = stopwords.words('spanish')
english_stopwords = stopwords.words('english')
#%%
mainpath = "C:/Users/guill/Documents/Universidad/PlataformaRefugiados/NAUTIA/DesarrolloPy/DataSetOriginales"

def fixBibliography(df):
    df = dfFix(df,"GENERAL INFORMATION - COUNTRY LEVEL")
    df.columns = ['GeneralInfo', 'CommunityCountry', 'RefugeeCountry']
    df.set_index('GeneralInfo', inplace = True)
    df = df.transpose()
    df = df.reset_index()
    return df

def dfFix(df,col1 = False,col2 = False):
    result = df.copy()
    if(col1):
        x = result.columns.get_loc(col1)
        result.drop(result.columns[0:x],axis = 1, inplace = True)
    if(col2):
        y = result.columns.get_loc(col2)
        result.drop(result.columns[y:],axis = 1, inplace = True)
    return result

def validColumn(cad):
    result = False
    if(cad == "index"):#Population
        result = True
    else:
        if(cad == "Type_of_settlement"):
            result = True
        else:
            if(cad == "General:settlement"):
                result = True
            else:
                if(cad == "general_info:_1_1_Choose_the_settlement"):
                    result = True
                else:
                    if(cad == "General_Information:Type_of_setlement"):
                        result = True
                    else:
                        if(cad == "General:Settlement"):
                            result = True
                        else:
                            if(cad == "Type_of_setlement"):
                                result = True
    return result

def dropRow(df,i):
    return df.drop(index = i)

def get_communityRows(df,cad,communityType): #la función pd.loc[] tiene un bug indiscriminado (https://github.com/pandas-dev/pandas/issues/8555)
    result = df
    if(communityType == 0):
        comm = "host_community"
    else:
        comm = "refugee_camp"
    for index, row in df.iterrows():
        if(row[cad] != comm):
            result = dropRow(result,index)
    return result

def setDataByIndex(df,communityType):
    array = df.columns
    i = 0
    df = df.replace("refugee","refugee_camp")
    df = df.replace("host_comunity","host_community")
    df = df.replace("RefugeeCountry","refugee_camp")
    df = df.replace("CommunityCountry","host_community")
    while(validColumn(array[i]) == False):
        i += 1
    df = get_communityRows(df,array[i],communityType)
    return df

def set_AllCSVtoDF(communityType):
    Bibliography = pd.read_excel(getPath(mainpath,"Bibliography_120220.xlsx"))
    Bibliography = fixBibliography(Bibliography)
    Bibliography = setDataByIndex(Bibliography,communityType)
    Entities = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Entities_Interview_results.csv")),communityType)
    LocalLeaders = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Local_leaders_v3_results.csv")),communityType)
    HouseHold = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Survey_household_v6_results.csv")),communityType)
    WomenGroup = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Women_Focus_Group2_results.csv")),communityType)
    SanitationInfra = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_V1_0_Sanitation_Infrastructre_results.csv")),communityType)
    Priorities = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Priorities_v3_results.csv")),communityType)
    GeneralForm = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_General_form_v3_results.csv")),communityType)
    PublicSpace = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Public_Space_results.csv")),communityType)
    WaterInf = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Water_Infrastructure_results.csv")),communityType)
    SanitationInf = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_V1_0_Sanitation_Infrastructre_results.csv")),communityType)
    WasteManagementInf = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Waste_Management_Infrastructure_results.csv")),communityType)
    EnergyINF = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Energy_Infrastructure_results.csv")),communityType)
    Business = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA1_0_Business_surveys_v3_results.csv")),communityType)
    MobilityINF = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0__Transport_servicesaccess_points_results.csv")),communityType) 
    ComunalServices = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Communal_Services_results.csv")),communityType) 
    GeneralCitizen = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_General_Citizen_Focus_Group_results.csv")),communityType)
    Shelter = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Shelter_results.csv")),communityType)
    FarmyardCrop = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Farmyard_and_Crops_results.csv")),communityType)
    return Bibliography,Entities,LocalLeaders,HouseHold,WomenGroup,SanitationInfra,Priorities,GeneralForm,PublicSpace,WaterInf,EnergyINF,SanitationInf,WasteManagementInf,EnergyINF,Business,MobilityINF,ComunalServices,GeneralCitizen,Shelter,FarmyardCrop

def concatDF(df1,df2):
    return  pd.concat([df1,df2],axis = 1, ignore_index = True, sort = True)

def mkCSV(df,fileName):
    df = df.dropna(how = 'all')
    df *= 1   
    fileName = fileName.lower()
    df.to_csv('DataSetFinales/'+fileName,sep=',',header = False, index=False, encoding='utf-8')
    
def getPath(mainpath,filename):
    return os.path.join(mainpath, filename)

def getSubColumnNames(df,x):
    columns = df.columns
    array = []
    for column in columns:
        column = column[x:]
        array.append(column)
    return pd.DataFrame(array) 

def addInstitutionAndType(df,array1,array2,instType):
    df = df.dropna(axis = 1)
    df = np.array(df)
    for row in df:
        for elem in row:
            array1 = np.append(array1,elem)
            array2 = np.append(array2,instType)
    return array1,array2

def politicalActor(df1,df2,df3,df4,df5):
    institution = []
    instType = []

    institution, instType  = addInstitutionAndType(df1,institution,instType,'Public Institution')
    institution, instType  = addInstitutionAndType(df2,institution,instType,'Private Institution')
    institution, instType  = addInstitutionAndType(df3,institution,instType,'NGO')
    institution, instType  = addInstitutionAndType(df4,institution,instType,'International Agency')
    institution, instType  = addInstitutionAndType(df5,institution,instType,'Local')

    institution = pd.DataFrame(institution)
    institution = institution.reset_index(drop = True)
    instType = pd.DataFrame(instType)
    instType = instType.reset_index(drop = True)
    
    return concatDF(institution,instType)


def get_claveValor(df1,df2,df3):
    array1 = np.array(df2)
    array2 = np.array(df1)
    result1 = []
    result2 = []
    result3 = []
    i = 0
    for row in array1:
        var = array2[i]
        for elem in row:
            result1 = np.append(result1,elem)
            result2 = np.append(result2,var)
        for r in np.array(df3):
            for l in r:
                result3 = np.append(result3,l)
        i+=1
    result2 = pd.DataFrame(result2)
    result2 = result2.reset_index(drop = True)
    result1 = pd.DataFrame(result1)
    result1 = result1.reset_index(drop = True)
    result3 = pd.DataFrame(result3)
    result3 = result3.reset_index(drop = True)
    return concatDF(result2,concatDF(result1,result3))

def get_number(df,df2):
    df = np.array(df)
    df2 = np.array(df2)
    array = np.array([])
    array2 = np.array([])
    for row in df:
        for elem in row:
            array = np.append(array,elem)
        for row2 in df2:
            for elem2 in row2:
                array2 = np.append(array2,elem2)
    array = (pd.DataFrame(array)).fillna(0) 
    array2 = pd.DataFrame(array2)
    return concatDF(array,array2)

def get_valueBySector(df1,df2):
    df2 = df2.reset_index()
    array1 = np.array(df1)
    i = 0
    for row in array1:
        for elem in row:
            if(elem == False):
                df2 = dropRow(df2,i)
        i += 1
    df2 = df2.set_index('index')
    return df2


def separateValues(df):
    if(df.isnull().values.all(axis=0)):
        result = df
    else:
        array = np.array(df)
        count_vectorizer = CountVectorizer(stop_words = spanish_stopwords+english_stopwords)
        corpus = []
        for row in array:
            for elem in row:
                corpus = np.append(corpus,[elem])
        X = count_vectorizer.fit_transform(corpus)
        array = count_vectorizer.get_feature_names()
        result = pd.DataFrame(array)
    return result

def vectorizeValue(df):
    season = pd.DataFrame()
    year = ['january','february','march','april','may','june','july','august','september','october','november','december']
    for row in np.array(df):
        corpus = np.array([])
        count_vectorizer = CountVectorizer(stop_words = spanish_stopwords+english_stopwords)
        for elem in row:
            corpus = np.append(corpus,[elem])
        X = count_vectorizer.fit_transform(corpus)
        array = count_vectorizer.get_feature_names()
        season = concatDF(season,pd.DataFrame(array))
    season = season.T
    result = pd.DataFrame()
    for row in np.array(season):
        vect = np.array([],dtype = bool)
        for elem in year:
            flag = False
            for month in row:
                if(month == elem):
                    flag = True
            if(flag):
                vect = np.append(vect,True)
            else:
                vect = np.append(vect,False)
        result = concatDF(result, pd.DataFrame(vect))
    return result.T



def set_sector(df,sect, concat = True):
    sector = np.array([])
    df = df.dropna(how = 'all')
    df = np.array(df)
    for column in df:
        sector = np.append(sector,sect)
    sector = pd.DataFrame(sector)
    df = pd.DataFrame(df)
    if(concat):
        result = concatDF(sector,df)
    else:
        result = sector
    return result 

def dropNaAndResetIndex(df):
    df = df.dropna(how = 'all')
    df = np.array(df)
    return pd.DataFrame(df)
    

def get_applianceDF(df):
    df1 = dfFix(df,0,1)
    df2 = dfFix(df,1)
    array = np.array([])
    appliance = np.array([])
    for row in np.array(df1):
        corpus = np.array([])
        for elem in row:
            corpus = np.append(corpus,[elem])
        count_vectorizer = CountVectorizer(stop_words = spanish_stopwords+english_stopwords, vocabulary = ['light_bulbs','mobile_phone','radio_music_pl','tv_dvd','laptop_tablet_','fridge','electrical_sto','others'])
        X = count_vectorizer.fit_transform(corpus)
        array = count_vectorizer.get_feature_names()
        for elem in array:
            appliance = np.append(appliance,elem)
    appliance = pd.DataFrame(appliance)
    hours = np.array([])
    for row in np.array(df2):
        for elem in row:
            hours = np.append(hours,elem)
    hours = pd.DataFrame(hours)
    hours = (hours.dropna())
    return concatDF(appliance,hours)
    

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

In [2]:
Bibliography,Entities,LocalLeaders,HouseHold,WomenGroup,SanitationInfra,Priorities,GeneralForm,PublicSpace,WaterInf,EnergyINF,SanitationInf,WasteManagementInf,EnergyINF,Business,MobilityINF,ComunalServices,GeneralCitizen,Shelter,FarmyardCrop = set_AllCSVtoDF(1)

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
Bibliography.shape

(1, 192)

In [5]:
Entities

,start,end,Type_of_settlement,Type_Host_Community,Identification:Institution_Name,Identification:Contact_person,Identification:Work_Area,GENERAL_INFORMATION:Secondary_movement,GENERAL_INFORMATION:Relationship,GENERAL_INFORMATION:Movement_outside,GENERAL_INFORMATION:Distance,Population:Women:Infants,Population:Women:Childs,Population:Women:Adults,Population:Women:Olders,Population:Men:Infants_001,Population:Men:Childs_001,Population:Men:Adults_001,Population:Men:Olders_001,Fuel_Cost:Main_Fuel,Fuel_Cost:Fuel_Cost_Diesel,Fuel_Cost:Fuel_Cost_Kerosene,Fuel_Cost:Fuel_Cost_Ethanol,Fuel_Cost:Fuel_Cost_Gas_cylder,ENERGY:Electricity_network,ENERGY:Covered_services,ENERGY:Power_failure,ENERGY:Demand_Curve,ENERGY:Street_Light,Urban_Planning_001:Urban_Planning,Urban_Planning_001:Period,Urban_Planning_001:Growth_area,Urban_Planning_001:Forecast,Urban_Planning_001:Land_Managment,Urban_Planning_001:Risk_Managment,Urban_Planning_001:Risk_Identified,Shelter:Plot_Area,Shelter:Plot_Limits,Shelter:Housing_Improvement,Shelter:Total_shelter,Shelter:Cost,Shelter:Shelter_Construction,Shelter:Vulnerable_Area:Vunerable_Area,Enviormental_Issues:Risk:Risk_Flood,Enviormental_Issues:Risk:Risk_dryness,Enviormental_Issues:Risk:Risk_earthquake,Enviormental_Issues:Risk:Risk_storm,Enviormental_Issues:Risk:Risk_plage,Enviormental_Issues:Deforestation,Enviormental_Issues:High_enviormental_value,Enviormental_Issues:Native_Plant,Enviormental_Issues:Native_Crops,Water_table,Sanitation:Personal_hygiene,Sanitation:Excreta,Sanitation:Open_defecation,Sanitation:Type_of_Latrine,Sanitation:Individual_Latrines,Waste_Managment:Waste_Collection,Waste_Managment:Landfill_Details:Location_1,Waste_Managment:Landfill_Details:Location_2,Waste_Managment:Landfill_Details:Adecuate_land,Water:Quality,Water:Treatment,Water:Comsuption,Water:Distribution,Water:Time,Women_Patrol,Education_Issues,Data_Access,Antenna,meta:instanceID
0,2020-03-26 15:15:35.02,2020-03-26 15:22:28.866,refugee_camp,NaN,ARRA,Mohammed,WASH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,vegetal_waste,yes,comunal,NaN,2.0,800_1000,closer,no,yes,NaN,10_20,improved__well,more_than_30_m,NaN,NaN,NaN,NaN,uuid:682e6419-b9fd-499f-b2a5-5fff7e145212
2,2020-03-26 15:26:43.124,2020-03-26 15:55:14.709,refugee_camp,NaN,IRC,Daniel Ruiz. Camp manager,manager_entity WASH food_security shelter energy,500.0,comercial,no,NaN,2000.0,3000.0,2500.0,500.0,NaN,NaN,NaN,NaN,ethanol,NaN,NaN,25.0,NaN,no,NaN,6.0,available,no,No,NaN,yes,4.0,private,yes,flood,10.0,no,no,1500.0,10.0,Humanitarian,Communal bathroom,25.0,20.0,1.0,60.0,1.0,1000.0,no,Árbol de coral,Teff,25.0,1.0,vegetal_waste,no,comunal,NaN,1.0,far_away,closer,yes,yes,Potabilzación,10_20,improved__well,more_than_30_m,no,long_distance_,mobile,1.0,uuid:2e6da402-c280-4d16-b2d1-8fa0decf11da
4,2020-03-27 12:36:03.121,2020-03-27 13:00:37.36,refugee_camp,NaN,Entidad 1,Persona 1,manager_entity WASH food_security shelter energy,100.0,comercial,yes,70.0,10000.0,15000.0,20000.0,30000.0,10000.0,15000.0,20000.0,30000.0,diesel,5.0,NaN,NaN,NaN,yes,educartional Kitchen bussiness hospital shelte...,1.0,not_available,no,No,NaN,yes,20.0,private,no,NaN,100.0,yes,yes,40000.0,1000.0,other,Sand storms,0.0,10.0,0.0,5.0,0.0,0.0,no,Phoenix dactylifera,No one,90.0,1.0,paper,no,individual,Yes,4.0,far_away,far_away,no,no,Underground water,5_10,lorries,more_than_30_m,no,economic_issue long_distance_ work illness other,mobile,5.0,uuid:51ad99c7-20ad-478f-be71-c7227fb698a7


In [6]:
LocalLeaders.shape

(3, 22)

In [7]:
HouseHold

,start,end,General:Gender,General:Settlement,General:Name_HC,General:old_years,General:Old_women,General:Young_wome,General:Old_men,General:Young_men,General:Children,Shelter:No_Rooms,Shelter:Improve_sh,Energy:Access_Y_N_001,Energy:Access_electric,Energy:Hours_day,Energy:HoursNight,Energy:Appliances,Energy:Hours_lightbulbs,Energy:Hours_mobile,Energy:Hours_radio_musicplayer,Energy:Hours_TV_DVD,Energy:Hours_laptop_computer_tablet,Energy:Hours_fridge,Energy:Hours_electrical_stove,Energy:Hours_others,Energy:Elec_expen,Energy:Solar_cost,Water:Water_col,health_001:Healthcare,Economy:FamilyHead,Economy:FamilyHead_001,Economy:Main_inco,Economy:Money,Economy:Food,Economy:clothes,Economy:water_001,Economy:education,Economy:transport,Economy:Health,Economy:firewood,meta:instanceID
0,2019-11-16 11:42:25.849,2019-11-18 23:38:40.155,female,refugee_camp,NaN,24.0,2.0,0.0,0.0,0.0,0.0,3,roofs,yes,electrical_gri_1,12.0,12.0,lantern light_bulbs mobile_phone tv/dvd fridge...,4.0,4.0,NaN,1.0,NaN,24.0,NaN,7.0,0.0,NaN,NaN,yes,no,female,others,10000.0,NaN,3000.0,0.0,0.0,5000.0,0.0,0.0,uuid:0e745beb-1ab2-43f6-b305-0282ed6f8625
1,2019-11-16 13:07:43.275,2019-11-16 13:20:14.748,female,refugee_camp,NaN,46.0,1.0,1.0,0.0,1.0,0.0,2,walls,yes,electrical_gri_1,12.0,12.0,light_bulbs mobile_phone tv/dvd fridge,4.0,2.0,NaN,5.0,NaN,24.0,NaN,NaN,0.0,NaN,NaN,yes,yes,NaN,remittances,7000.0,5000.0,500.0,0.0,0.0,500.0,0.0,0.0,uuid:821bc2d2-3980-45c7-a360-191d5ad2e719
2,2019-11-26 10:12:03.488,2019-11-26 14:17:27.764,female,refugee_camp,NaN,50.0,3.0,0.0,2.0,NaN,1.0,2,roofs,yes,electrical_gri_1,12.0,12.0,lantern light_bulbs mobile_phone fridge,4.0,2.0,NaN,NaN,NaN,24.0,NaN,NaN,40.0,NaN,0.0,yes,yes,NaN,livestock,NaN,300.0,NaN,NaN,NaN,NaN,100.0,40.0,uuid:368a7f8a-af84-4bdc-a445-2a7c07419ce6
3,2019-11-26 10:40:08.897,2019-11-26 14:23:14.454,female,refugee_camp,NaN,68.0,41.0,NaN,NaN,NaN,NaN,1,roofs,yes,electrical_gri_1,12.0,12.0,lantern light_bulbs mobile_phone tv/dvd fridge,3.0,1.0,NaN,1.0,NaN,24.0,NaN,NaN,1000.0,NaN,NaN,yes,yes,NaN,subsidies,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,1000.0,uuid:7a1c1220-25e0-4721-900e-232c9322f2d7
4,2019-11-26 10:58:11.521,2019-11-26 14:24:25.824,male,refugee_camp,NaN,33.0,2.0,NaN,4.0,NaN,NaN,1,roofs,yes,electrical_gri_1,12.0,12.0,lantern light_bulbs mobile_phone tv/dvd fridge,4.0,1.0,NaN,2.0,NaN,24.0,NaN,NaN,1000.0,NaN,NaN,yes,no,male,commerce,5000.0,1000.0,NaN,NaN,NaN,1000.0,NaN,1000.0,uuid:78457e10-3e82-401f-96aa-76878e961852
5,2019-11-26 11:17:05.719,2019-11-26 14:25:24.879,female,refugee_camp,NaN,62.0,5.0,NaN,3.0,NaN,2.0,3,foundations,yes,electrical_gri_1,12.0,12.0,lantern light_bulbs mobile_phone radio/music_p...,4.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,1400.0,NaN,NaN,yes,yes,NaN,subsidies,1000.0,300.0,NaN,NaN,NaN,NaN,750.0,NaN,uuid:280883bf-7252-457c-85aa-ccae46fc97fe
6,2019-11-26 11:36:40.803,2019-11-26 14:26:12.67,female,refugee_camp,NaN,45.0,2.0,2.0,1.0,1.0,2.0,2,roofs,yes,electrical_gri_1,12.0,12.0,lantern light_bulbs mobile_phone radio/music_p...,6.0,2.0,3.0,10.0,NaN,24.0,NaN,NaN,1500.0,NaN,NaN,yes,no,male,others,1000.0,200.0,NaN,NaN,NaN,NaN,1200.0,1500.0,uuid:5e775105-7594-48f8-bfcd-b047953a218b
7,2019-11-26 11:53:12.636,2019-11-26 14:26:38.237,female,refugee_camp,NaN,65.0,2.0,NaN,2.0,NaN,NaN,2,foundations,yes,electrical_gri_1,12.0,12.0,lantern light_bulbs mobile_phone tv/dvd fridge,4.0,1.0,NaN,4.0,NaN,24.0,NaN,NaN,1000.0,NaN,NaN,yes,yes,NaN,subsidies,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,uuid:4ebe49be-398a-4dea-96c9-da3e66605031
8,2019-11-26 12:23:42.578,2019-11-26 12:31:38.046,female,refugee_camp,NaN,60.0,2.0,NaN,2.0,NaN,NaN,2,roofs,yes,electrical_gri_1,12.0,12.0,lantern light_bulbs tv/dvd fridge,4.0,NaN,NaN,3.0,NaN,24.0,NaN,NaN,NaN,NaN,NaN,yes,yes,NaN,others,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,uuid:3d550f75-60e7-4f47-b5ed-57bcea6b6a8d
9,2019-11-26 12:42:13.702,2019-11-26 12:47:03.621,female,refugee_camp,NaN,24.0,1.0,NaN,1.0,NaN,2.0,3,nothing,yes,electrical_gri_1,12.0,12.0,lantern light_bulbs mobile_phone tv/dvd fridge,4.0,1.0,NaN,1.0,NaN,24.0,Na

In [8]:
WomenGroup.shape

(1, 27)

In [9]:
SanitationInfra.shape

(37, 14)

In [10]:
Priorities

,start,end,Type_of_setlement,Type_of_Host_Community,Participants,Participants_001,Participants_002,group_yf0yl72:Instruction_1,group_yf0yl72:Energy_1,group_yf0yl72:Shelter_1,group_yf0yl72:Water_Access_1,group_yf0yl72:Sanitation_1,group_yf0yl72:Education_1,group_yf0yl72:Health_1,group_yf0yl72:Public_space_1,group_yf0yl72:Food_1,group_yf0yl72:TICS_1,group_yf0yl72:Work_1,group_yf0yl72:Waste_Managment,group_yf0yl72:Public_Transport_001,group_yf0yl72:Religious_Center,group_yf0yl72:Socio_Cultural_Center,group_yf0yl72:Market,Priority_2:Instruction,Priority_2:Energy_2,Priority_2:Shelter_2,Priority_2:Water_Access_2,Priority_2:Sanitation_2,Priority_2:Education_2,Priority_2:Health_2,Priority_2:Public_space_2,Priority_2:Food_2,Priority_2:TICS_2,Priority_2:Work_2,Priority_2:Waste_Managment_2,Priority_2:Public_Transport_2,Priority_2:Religious_Center_2,Priority_2:Socio_Cultural_Center_2,Priority_2:Market_2,Priority_3:Instruction_001,Priority_3:Energy_3,Priority_3:Shelter_3,Priority_3:Water_Access_3,Priority_3:Sanitation_3,Priority_3:Education_3,Priority_3:Health_3,Priority_3:public_space_3,Priority_3:Food_3,Priority_3:TICS_3,Priority_3:Work_3,Priority_3:Waste_Managment_3,Priority_3:Public_Transport,Priority_3:Socio_Cultural_Center_3_001,Priority_3:Religious_Center_3_001,Priority_3:Market_001,Priority_4:Instruction_002,Priority_4:Energy_4_001,Priority_4:Shelter_4_001,Priority_4:Water_Access_4_001,Priority_4:Sanitation_4_001,Priority_4:Education_4_001,Priority_4:Health_4_001,Priority_4:public_space_4,Priority_4:Food_4,Priority_4:TICS_4,Priority_4:Work_4,Priority_4:Market_4,Priority_4:Religious_Center_4_001,Priority_4:Socio_Cultural_Center_4_001,Priority_4:Public_Transport_4,Priority_4:Waste_Managment_4,Priority_5:Instruction_003,Priority_5:Energy_4,Priority_5:Shelter_4,Priority_5:Water_Access_4,Priority_5:Sanitation_4,Priority_5:Education_4,Priority_5:Health_4,Priority_5:public_space_4_001,Priority_5:Food_4_001,Priority_5:TICS_5,Priority_5:Work_5,Priority_5:Market_5,Priority_5:Religious_Center_5,Priority_5:Socio_Cultural_Center_5,Priority_5:Public_Transport_5,Priority_5:Waste_Managment_5,meta:instanceID
0,2019-11-09 11:55:45.659,2019-11-09 12:02:08.443,refugee_camp,NaN,19.0,NaN,NaN,NaN,0.0,16.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,7.0,3.0,1.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,0.0,10.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,NaN,3.0,0.0,5.0,0.0,2.0,6.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,2.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,6.0,0.0,1.0,0.0,3.0,1.0,uuid:e84fd6bd-e2ba-4578-a333-fb234229b9af
1,2020-03-24 12:35:14.031,2020-03-24 13:20:13.955,refugee_camp,NaN,50.0,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,20.0,5.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,3.0,10.0,2.0,2.0,2.0,3.0,3.0,2.0,3.0,4.0,7.0,2.0,5.0,3.0,NaN,2.0,2.0,2.0,2.0,10.0,1.0,2.0,10.0,10.0,1.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,2.0,15.0,15.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,NaN,2.0,0.0,2.0,2.0,2.0,15.0,2.0,15.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,uuid:92e5d2bb-9201-49d9-9ebe-e153d02ad3ab


In [11]:
GeneralForm.shape

(1, 19)

In [12]:
PublicSpace

,start,end,Public_Space,Type_of_setlement,Type_of_Host_Community,Record_your_current_location:Latitude,Record_your_current_location:Longitude,Record_your_current_location:Altitude,Record_your_current_location:Accuracy,Details:Shade_Areas,Details:Urban_Furniture,Details:Paved_Area,Details:Waste_Managment,meta:instanceID
0,2019-11-21 10:37:17.2,2019-11-22 23:16:50.465,NaN,refugee_camp,NaN,27.490511,-7.829464,458.814941,4.7000000000,no,no,yes,no,uuid:84bb73a3-b5ce-4f44-9eef-0571c95af66c
1,2020-03-26 11:14:46.465,2020-03-26 11:19:35.276,NaN,refugee_camp,NaN,40.386820,-3.699133,0.000000,18.2460000000,yes,yes,yes,yes,uuid:218d00b1-761d-401e-876a-508ce14ffd8d


In [13]:
WaterInf.shape

(30, 13)

In [14]:
SanitationInf.shape

(37, 14)

In [15]:
WasteManagementInf.shape

(1, 10)

In [16]:
EnergyINF.shape

(4, 38)

In [17]:
Business.shape

(16, 25)

In [18]:
MobilityINF.shape

(3, 10)

In [19]:
ComunalServices.shape

(43, 63)

In [20]:
GeneralCitizen.shape

(1, 43)

In [21]:
Shelter.shape

(23, 27)

In [22]:
FarmyardCrop.shape

(16, 18)

In [23]:
PublicSpace.shape

(2, 14)

In [24]:

def set_defaultColumn(df):
    default = np.array([])
    for row in df:
        default = np.append(default,np.nan)
    default = pd.DataFrame(default)
    return concatDF(df,default)        
        
def replacestr(df,cad1,cad2):
    cols=list(df.columns)
    for col in cols:
        df[col] = df[col].astype(str).str.replace(cad1,cad2)
    return df


In [81]:
u_area = dfFix(PublicSpace,"Details:Shade_Areas","meta:instanceID")
u_area = getSubColumnNames(u_area,8)
u_area = pd.DataFrame(u_area)
mkCSV(u_area,"u_area.csv")

df1 = dfFix(PublicSpace,"Details:Shade_Areas","meta:instanceID")
df1 = df1.isin(["yes"])
u_area = getSubColumnNames(df1,8)
df = pd.DataFrame()
U_Area_has_Community = dfFix(PublicSpace,"Record_your_current_location:Latitude","Record_your_current_location:Accuracy")
j = 0
for row in np.array(df1):
    i = 0
    array = np.array([],dtype = int)
    array2 = np.array([])
    for elem in row:
        if(elem == True):
            array = np.append(array,i)
        i += 1
    for i in array:
        array2 = np.append(array2,np.array(u_area)[i])
    df = concatDF(df,pd.DataFrame(array2))
    j += 1
U_Area_has_Community = concatDF(U_Area_has_Community,df.T)
df = pd.DataFrame()
for row in np.array(U_Area_has_Community):
    array = np.array([])
    for elem in row[3:]:
        if(str(elem) != 'nan'):
            array = np.append(array,elem)
    for elem in array:
        array2 = row[:3]
        array2 = np.append(array2,elem)
        df = concatDF(df,pd.DataFrame(array2))
U_Area_has_Community = df.T
mkCSV(U_Area_has_Community,"U_Area_has_Community.csv")


In [82]:
U_Area_has_Community

,0,1,2,3
0,27.4905,-7.82946,458.815,Paved_Area
1,40.3868,-3.69913,0,Shade_Areas
2,40.3868,-3.69913,0,Urban_Furniture
3,40.3868,-3.69913,0,Paved_Area
4,40.3868,-3.69913,0,Waste_Managment


In [79]:
df

,0,1,2,3
0,27.4905,-7.82946,458.815,Paved_Area
1,40.3868,-3.69913,0,Shade_Areas
2,40.3868,-3.69913,0,Urban_Furniture
3,40.3868,-3.69913,0,Paved_Area
4,40.3868,-3.69913,0,Waste_Managment


In [27]:
u_area

,0
0,Shade_Areas
1,Urban_Furniture
2,Paved_Area
3,Waste_Managment
